- #### Transformaciones
    - ##### select - when
    - ##### where/filter


In [1]:
import org.apache.spark.sql.{SparkSession, DataFrame, Column}

val spark = SparkSession.builder
        .appName("sesion_1")
        .master("local[*]")
        .getOrCreate()
val sc = spark.sparkContext

sc = org.apache.spark.SparkContext@e6be2d0


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5b7a3662


org.apache.spark.SparkContext@e6be2d0

In [2]:
sc.uiWebUrl

Some(http://23LAP5CD20860NP.indra.es:4041)

In [3]:
def readCsv(path: String): DataFrame = {
    spark.read
        .option("header","true")
        .option("delimiter",",")
        .option("inferSchema","false")
        .csv(path)
}

val BasePath = "../../resources/data/csv/"
val contractsDf = readCsv(BasePath + "contracts.csv")

contractsDf.show()

contractsDf = [cod_iuc: string, cod_titular: string ... 3 more fields]


readCsv: (path: String)org.apache.spark.sql.DataFrame
BasePath: String = ../../resources/data/csv/


+-------+-----------+------------+----------+------+
|cod_iuc|cod_titular|cod_producto|  fec_alta|activo|
+-------+-----------+------------+----------+------+
|  30000|      00006|         100|2012-05-01|  true|
|  30001|      00006|         200|2014-05-01|  true|
|  30002|      00006|         300|2006-02-01| false|
|  30003|      00006|         150|2012-05-01|  true|
|  30002|      00005|         300|2012-05-01|  true|
|  30004|      00006|         400|2012-05-01| false|
|  30005|      00006|         500|2012-05-01|  true|
|  30006|      00006|         600|2012-05-01| false|
|  30003|      00003|         150|2019-10-14|  true|
|  30007|      00006|         700|2014-02-01| false|
|  30008|      00006|         800|2012-05-01|  true|
|  30009|      00006|         900|2015-09-01|  true|
|  30009|      00002|         900|2009-10-01|  true|
|  30010|      00006|        1000|2014-02-01|  true|
|  30003|      00002|         150|2018-09-18|  true|
|  30011|      00003|        1100|2018-10-01| 

[cod_iuc: string, cod_titular: string ... 3 more fields]

In [6]:
import org.apache.spark.sql.{functions => f}
import org.apache.spark.sql.{types => t}

// Select

def difference(l1: Seq[String], l2: Seq[String]): Seq[Column] = {
    l1.diff(l2).map(colName => f.col(colName))
}
// Concatenar :+
// Desempaquetar :_*
val resultedDf = contractsDf
    .select(
        difference(contractsDf.columns.toSeq, Seq("fec_alta", "activo")) :+
        f.col("activo").cast(t.BooleanType) :+
        f.col("fec_alta").alias("fec_alta_ini").cast(t.DateType) :+
        f.current_date().alias("actual_date") :+
        f.round(f.rand(0) * f.lit(10)).cast(t.IntegerType).alias("randm_num") :+
        f.date_add(f.col("fec_alta_ini"), f.col("randm_num")).alias("fec_alta_fin") :+
        f.datediff(f.col("fec_alta_fin"), f.col("fec_alta_ini")).alias("diff") :_*
        )
    .drop("randm_num")

resultedDf.show()
resultedDf.printSchema()

resultedDf = [cod_iuc: string, cod_titular: string ... 6 more fields]


import org.apache.spark.sql.{functions=>f}
import org.apache.spark.sql.{types=>t}
difference: (l1: Seq[String], l2: Seq[String])Seq[org.apache.spark.sql.Column]


+-------+-----------+------------+------+------------+-----------+------------+----+
|cod_iuc|cod_titular|cod_producto|activo|fec_alta_ini|actual_date|fec_alta_fin|diff|
+-------+-----------+------------+------+------------+-----------+------------+----+
|  30000|      00006|         100|  true|  2012-05-01| 2024-02-22|  2012-05-09|   8|
|  30001|      00006|         200|  true|  2014-05-01| 2024-02-22|  2014-05-06|   5|
|  30002|      00006|         300| false|  2006-02-01| 2024-02-22|  2006-02-02|   1|
|  30003|      00006|         150|  true|  2012-05-01| 2024-02-22|  2012-05-04|   3|
|  30002|      00005|         300|  true|  2012-05-01| 2024-02-22|  2012-05-08|   7|
|  30004|      00006|         400| false|  2012-05-01| 2024-02-22|  2012-05-04|   3|
|  30005|      00006|         500|  true|  2012-05-01| 2024-02-22|  2012-05-03|   2|
|  30006|      00006|         600| false|  2012-05-01| 2024-02-22|  2012-05-07|   6|
|  30003|      00003|         150|  true|  2019-10-14| 2024-02-22

[cod_iuc: string, cod_titular: string ... 6 more fields]

In [7]:
// select- when

val cond_1: Column = f.col("cod_producto") <= 300 // baja
val cond_2: Column = f.col("cod_producto") <= 600 // media
val cond_3: Column = f.col("cod_producto") <= 1000 // alta

val selectWhenDf = resultedDf
    .select(
        difference(resultedDf.columns.toSeq, Seq("activo")) :+
        f.when(cond_1, f.lit("baja"))
            .when(cond_2, f.lit("media"))
            .when(cond_3, f.lit("alta"))
            .otherwise(f.lit("muy alta")).alias("calidad") :+
        f.when((f.col("activo") === f.lit(true)) && (f.col("calidad").isin("alta", "muy alta")), f.lit("ok")).alias("prioridad_alta") :+
        f.when(f.col("activo") === true, f.lit(true)).alias("activo") :_*
    )

selectWhenDf.show()

selectWhenDf = [cod_iuc: string, cod_titular: string ... 8 more fields]


cond_1: org.apache.spark.sql.Column = (cod_producto <= 300)
cond_2: org.apache.spark.sql.Column = (cod_producto <= 600)
cond_3: org.apache.spark.sql.Column = (cod_producto <= 1000)


+-------+-----------+------------+------------+-----------+------------+----+--------+--------------+------+
|cod_iuc|cod_titular|cod_producto|fec_alta_ini|actual_date|fec_alta_fin|diff| calidad|prioridad_alta|activo|
+-------+-----------+------------+------------+-----------+------------+----+--------+--------------+------+
|  30000|      00006|         100|  2012-05-01| 2024-02-22|  2012-05-09|   8|    baja|          null|  true|
|  30001|      00006|         200|  2014-05-01| 2024-02-22|  2014-05-06|   5|    baja|          null|  true|
|  30002|      00006|         300|  2006-02-01| 2024-02-22|  2006-02-02|   1|    baja|          null|  null|
|  30003|      00006|         150|  2012-05-01| 2024-02-22|  2012-05-04|   3|    baja|          null|  true|
|  30002|      00005|         300|  2012-05-01| 2024-02-22|  2012-05-08|   7|    baja|          null|  true|
|  30004|      00006|         400|  2012-05-01| 2024-02-22|  2012-05-04|   3|   media|          null|  null|
|  30005|      0000

[cod_iuc: string, cod_titular: string ... 8 more fields]

In [8]:
// where / filter
// La negación en Scala es con !
selectWhenDf
    .filter(f.col("activo").isNotNull)
    .filter(!f.col("cod_producto").isin("100", "200", "150", "300"))
    .where(f.col("fec_alta_ini").between("2014-02-01", "2018-10-01"))
    .where((f.col("calidad") === "media") || (f.col("cod_titular").isin("00006", "00001")))
    .show()

+-------+-----------+------------+------------+-----------+------------+----+-------+--------------+------+
|cod_iuc|cod_titular|cod_producto|fec_alta_ini|actual_date|fec_alta_fin|diff|calidad|prioridad_alta|activo|
+-------+-----------+------------+------------+-----------+------------+----+-------+--------------+------+
|  30009|      00006|         900|  2015-09-01| 2024-02-22|  2015-09-07|   6|   alta|            ok|  true|
|  30010|      00006|        1000|  2014-02-01| 2024-02-22|  2014-02-06|   5|   alta|            ok|  true|
|  30007|      00001|         700|  2016-07-01| 2024-02-22|  2016-07-03|   2|   alta|            ok|  true|
|  30006|      00005|         600|  2016-12-10| 2024-02-22|  2016-12-10|   0|  media|          null|  true|
+-------+-----------+------------+------------+-----------+------------+----+-------+--------------+------+



In [9]:
selectWhenDf
    .write.mode("overwrite").parquet("../../resources/data/parquet/contracts_tmp")